In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import datetime as dt

In [3]:
import sys
sys.path.append("../") 

from utils.paths import make_dir_line

modality = 'e'
project = 'agrupaciones'
data = make_dir_line(modality, project)

raw = data('raw')
processed = data('processed')

# AIMS

1. Ratio de endeudamiento: ratio de la suma de las deudas sobre la suma de todo los creditos
2. Suma de cuota mensual monetaria por cada credito: cuota mensual por persona
3. Suma de creditos por mes: 
4. Razon de uso de credito

## 1. Ratio de endeudamiento por cliente general

In [4]:
creditos = pd.read_csv(processed / 'creditos.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
clientes_agrupados = creditos.groupby('id_cliente').sum()
clientes_agrupados

C:\Users\Oscar\AppData\Local\Temp\ipykernel_6944\2009055674.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  clientes_agrupados = creditos.groupby('id_cliente').sum()


,id_prestamo,monto,cuotas,deuda_actual,dias_atraso
id_cliente,,,,,
1,40017,650,22,241,4
2,40019,650,28,109,18


In [5]:
clientes_agrupados['ratio_endeudamiento'] = clientes_agrupados['deuda_actual'] / clientes_agrupados['monto']
clientes_agrupados

,id_prestamo,monto,cuotas,deuda_actual,dias_atraso,ratio_endeudamiento
id_cliente,,,,,,
1,40017,650,22,241,4,0.370769
2,40019,650,28,109,18,0.167692


## 2. Suma de cuota mensual monetaria por cada credito

In [6]:
# funcion mes
def get_month(x):
    return dt.datetime(x.year, x.month, 1)

In [7]:
creditos = pd.read_parquet(processed / 'compras.parquet.gzip')
creditos['fecha_mes'] = creditos['fecha'].apply(get_month)
creditos.set_index("fecha_mes", inplace=True)
creditos.head()

,cliente,monto,cuotas,fecha,dias_mora
fecha_mes,,,,,
2022-01-01,Ana,2000,6,2022-01-01,25
2022-03-01,Ana,3000,12,2022-03-01,0
2022-03-01,Ana,5000,24,2022-03-01,8
2023-02-01,Ana,300,2,2023-02-01,7
2022-01-01,Juan,1000,12,2022-01-01,0


In [8]:
creditos["cuota_mensual"] = creditos["monto"] / creditos["cuotas"]
creditos

,cliente,monto,cuotas,fecha,dias_mora,cuota_mensual
fecha_mes,,,,,,
2022-01-01,Ana,2000,6,2022-01-01,25,333.333333
2022-03-01,Ana,3000,12,2022-03-01,0,250.000000
2022-03-01,Ana,5000,24,2022-03-01,8,208.333333
2023-02-01,Ana,300,2,2023-02-01,7,150.000000
2022-01-01,Juan,1000,12,2022-01-01,0,83.333333
2022-02-01,Juan,5000,24,2022-02-01,0,208.333333
2022-01-01,Pedro,1500,8,2022-01-01,11,187.500000
2022-02-01,Pedro,2000,12,2022-02-01,0,166.666667


In [9]:
deuda_mensual = creditos.groupby([pd.Grouper(freq="M"), "cliente"])["cuota_mensual"].sum()
deuda_mensual = deuda_mensual.reset_index()
deuda_mensual

,fecha_mes,cliente,cuota_mensual
0,2022-01-31,Ana,333.333333
1,2022-01-31,Juan,83.333333
2,2022-01-31,Pedro,187.500000
3,2022-02-28,Juan,208.333333
4,2022-02-28,Pedro,166.666667
5,2022-03-31,Ana,458.333333
6,2023-02-28,Ana,150.000000


## 3. Suma de creditos per month

In [10]:
# out = (creditos.assign(month=creditos['date'].dt.to_period('M'))
#         # duplicate rows corresponding to the duration
#         .loc[creditos.index.repeat(creditos['duration'])]
#         # increment months and ensure correct order
#         .assign(month=lambda d: d['month']+d.groupby(level=0).cumcount().add(1))
#         .sort_values(by=['id_customer', 'month'])
#         # cumulated sum of installments
#         .assign(total_sum=lambda d: d.groupby(level=0)['installment'].cumsum())
#         # sum per customer
#         .groupby(['id_customer', 'month'], as_index=False)
#         .agg(**{'total_sum': ('total_sum', 'sum'),
#                 'sum_cred': ('credit', 'sum'),
#                })
#         # computation of total_inv and shift of sum_cred
#         .assign(total_inv=lambda d: d['sum_cred'].sub(d['total_sum']),
#                 sum_cred=lambda d: d.groupby('id_customer')['sum_cred'].shift(-1, fill_value=0)
#                )
#        )

# out

In [11]:
creditos = pd.read_csv(processed / 'historial_compras.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
creditos['date'] = pd.to_datetime(creditos['date'], dayfirst='false', errors = 'coerce')
creditos

,id_customer,id_credit,duration,date,credit,installment,installment_amount
0,A,1001,4,2022-01-02,100,25,4.0
1,B,1002,3,2022-02-03,90,30,3.0
2,A,1003,5,2022-03-04,250,50,5.0


In [12]:
## Crea una columna llamada month en la cual se da entender el mes de toma del credito
creditos = creditos.assign(month=creditos['date'].dt.to_period('M'))
creditos

,id_customer,id_credit,duration,date,credit,installment,installment_amount,month
0,A,1001,4,2022-01-02,100,25,4.0,2022-01
1,B,1002,3,2022-02-03,90,30,3.0,2022-02
2,A,1003,5,2022-03-04,250,50,5.0,2022-03


In [13]:
## Repite la fila por la duracion, posteriormente le adiciona un mes por linea
creditos = creditos.loc[creditos.index.repeat(creditos['duration'])].assign(month=lambda d: d['month']+d.groupby(level=0).cumcount().add(1))
creditos

,id_customer,id_credit,duration,date,credit,installment,installment_amount,month
0,A,1001,4,2022-01-02,100,25,4.0,2022-02
0,A,1001,4,2022-01-02,100,25,4.0,2022-03
0,A,1001,4,2022-01-02,100,25,4.0,2022-04
0,A,1001,4,2022-01-02,100,25,4.0,2022-05
1,B,1002,3,2022-02-03,90,30,3.0,2022-03
1,B,1002,3,2022-02-03,90,30,3.0,2022-04
1,B,1002,3,2022-02-03,90,30,3.0,2022-05
2,A,1003,5,2022-03-04,250,50,5.0,2022-04
2,A,1003,5,2022-03-04,250,50,5.0,2022-05
2,A,1003,5,2022-03-04,250,50,5.0,2022-06


In [14]:
## Ordena el dataframe por customer and month
creditos = creditos.sort_values(by=['id_customer', 'month'])
creditos

,id_customer,id_credit,duration,date,credit,installment,installment_amount,month
0,A,1001,4,2022-01-02,100,25,4.0,2022-02
0,A,1001,4,2022-01-02,100,25,4.0,2022-03
0,A,1001,4,2022-01-02,100,25,4.0,2022-04
2,A,1003,5,2022-03-04,250,50,5.0,2022-04
0,A,1001,4,2022-01-02,100,25,4.0,2022-05
2,A,1003,5,2022-03-04,250,50,5.0,2022-05
2,A,1003,5,2022-03-04,250,50,5.0,2022-06
2,A,1003,5,2022-03-04,250,50,5.0,2022-07
2,A,1003,5,2022-03-04,250,50,5.0,2022-08
1,B,1002,3,2022-02-03,90,30,3.0,2022-03


In [15]:
## Crea la columna total_sum
creditos = creditos.assign(total_sum=lambda d: d.groupby(level=0)['installment'].cumsum())
creditos

,id_customer,id_credit,duration,date,credit,installment,installment_amount,month,total_sum
0,A,1001,4,2022-01-02,100,25,4.0,2022-02,25
0,A,1001,4,2022-01-02,100,25,4.0,2022-03,50
0,A,1001,4,2022-01-02,100,25,4.0,2022-04,75
2,A,1003,5,2022-03-04,250,50,5.0,2022-04,50
0,A,1001,4,2022-01-02,100,25,4.0,2022-05,100
2,A,1003,5,2022-03-04,250,50,5.0,2022-05,100
2,A,1003,5,2022-03-04,250,50,5.0,2022-06,150
2,A,1003,5,2022-03-04,250,50,5.0,2022-07,200
2,A,1003,5,2022-03-04,250,50,5.0,2022-08,250
1,B,1002,3,2022-02-03,90,30,3.0,2022-03,30


In [16]:
## Agrupa por customer and month
creditos = creditos.groupby(['id_customer', 'month'], as_index=False).agg(**{'total_sum': ('total_sum', 'sum'),
                                                                             'sum_cred': ('credit', 'sum'),})
creditos

,id_customer,month,total_sum,sum_cred
0,A,2022-02,25,100
1,A,2022-03,50,100
2,A,2022-04,125,350
3,A,2022-05,200,350
4,A,2022-06,150,250
5,A,2022-07,200,250
6,A,2022-08,250,250
7,B,2022-03,30,90
8,B,2022-04,60,90
9,B,2022-05,90,90


In [17]:
creditos = creditos.assign(total_inv=lambda d: d['sum_cred'].sub(d['total_sum']),
                           sum_cred=lambda d: d.groupby('id_customer')['sum_cred'].shift(-1, fill_value=0)
                          )
creditos

,id_customer,month,total_sum,sum_cred,total_inv
0,A,2022-02,25,100,75
1,A,2022-03,50,350,50
2,A,2022-04,125,350,225
3,A,2022-05,200,250,150
4,A,2022-06,150,250,100
5,A,2022-07,200,250,50
6,A,2022-08,250,0,0
7,B,2022-03,30,90,60
8,B,2022-04,60,90,30
9,B,2022-05,90,0,0


## 4. Razon de uso de credito

In [18]:
creditos2 = creditos.groupby('id_customer')['sum_cred'].max().reset_index()
creditos2

,id_customer,sum_cred
0,A,350
1,B,90


In [19]:
# redondear a multiplo de 10 es con -1,  100 es con -2, 1000 es con -3
creditos2['suma_redondeada'] = round(creditos2['sum_cred'], -2)
creditos2

,id_customer,sum_cred,suma_redondeada
0,A,350,400
1,B,90,100


In [20]:
print('ok_')

ok_
